In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import nltk
import time
from tqdm import tqdm
import pylab
import warnings
warnings.filterwarnings('ignore')

In [2]:
df = pd.read_csv("amazon_reviews.csv")

In [3]:
df.head()

,content
0,\nExcellent - the best thing that I’ve ever ta...
1,"\nWow, these are so addicting! I really don’t ..."
2,\nIt's hard to find quality snacks here. These...
3,\nWith healthy ingredients and strong flavor o...
4,\nWay more cheesy taste that the orange puffs!...


## POS Topics

In [4]:
from collections import Counter, defaultdict
import spacy
nlp = spacy.load("en_core_web_sm")

In [5]:
# Define subject-pronouns to remove
personal_pronouns = ['i', 'me', 'you', 'he', 'she', 'it', 'we', 'us']  # removed 'they', 'them'

In [6]:
def create_sub_phrases(X):
  """Returns lists of tuple phrases"""
  wave_1 = []
  wave_2 = []

  for i in tqdm(range(len(X))):
    sente = nlp(X.iloc[i])
    for token in sente:
      noun = ''
      adj = ''
      adverb = ''
      neg = ''
      adv_verb = ''
      subject = ''
      dobj_text = ''
      adj_text = ''
      if token.dep_ == 'ROOT':
        for child in token.children:
          if child.pos_ == 'NOUN':
            noun = child.text
          elif child.pos_ == 'ADJ':
            adj = child.text
            for other_child in child.children:
              if other_child.pos_ == 'ADV':
                adverb = other_child.text
          elif child.pos_ == 'ADV':
            adv_verb = child.text
          elif child.pos_ == 'PART':
            neg = child.text

          if child.dep_ in ['nsubj', 'nsubjpass'] and child.text.lower() not in personal_pronouns:  # nsubjpass for passive voice
            subject = child.text

          elif child.dep_ == 'dobj' and child.pos_ == 'NOUN':
            dobj_text = child.text
            for grandchild in child.children:
              if grandchild.dep_ == 'amod' and grandchild.pos_ == 'ADJ':
                  adj_text = grandchild.text

        if noun and adj:
          if adverb :
            wave_1.append((noun, token.text,adverb,adj))
          elif adv_verb and neg:
            wave_1.append((noun, token.text,adv_verb,neg,adj))
          elif neg:
            wave_1.append((noun, token.text,neg,adj))
          else:
            wave_1.append((noun, token.text,adj))

        if subject and dobj_text and adj_text:
          wave_2.append((subject, token.text, adj_text, dobj_text))

  return wave_1 + wave_2

def get_topics(phrases):
  """Returns list of top 5 topics"""
  topic_grams = []
  topics = []
  for phrase in phrases:
    if len(phrase) == 3:
      uni_gram = nlp(phrase[2])[0].lemma_
      topic_grams.append(uni_gram)
    if len(phrase) == 4:
      bi_gram = phrase[2] + " " + phrase[3]
      topic_grams.append(bi_gram)
  counter = Counter(topic_grams).most_common(5)
  for tuple in counter:
    topics.append(tuple[0])
  return topics

In [7]:
phrases = create_sub_phrases(df.content)
topics = get_topics(phrases)
topics

100%|██████████| 70/70 [00:01<00:00, 64.57it/s]


['great', 'delicious', 'nice', 'cheesy', 'so good']